# Sanity checking user input with `__init__`

The last unit concluded with a simple Python implementation of finite-state automata.
The code is repeated here (with shortened docstrings, for the sake of succinctness).

In [ ]:
class fsa:
    def __init__(self, initial=0, final=set(), transitions={}):
        """Class for finite-state automata.
        
        The transitions must be a dictionary of the form
        {current_state: {arc_label: new_state}}.
        """
        self.I = initial
        self.F = final
        self.T = transitions
        

    def accepts(self, sentence):
        """Test if FSA accepts sentence."""
        # set current state to initial state
        cs = self.I
        # iterate over sentence and follow along in automaton
        for word in sentence:
            cs = self.T.get(cs, {}).get(word)
            if cs is None:
                return False
        # did we make it to a final state?
        return True if cs in self.F else False

One downside to this implementation is that transitions must be passed in as a dictionary.
Don't get me wrong, we definitely want to **store** the transitions as a dictionary, it makes them very easy to work with.
But dictionaries aren't the best way to **specify** the transitions.
They are tedious to type, there's a lot of braces, commas, colons, and quotations that need to go in the right place.
Manually writing a dictionary is likely to lead to errors.
At the very least, our code should allow for an easier alternative.

## Specifying transitions as triples

Remember that the mathematical definition of an FSA states that the automaton's collection of transitions is a set of triples.
Each triple specifies

1. the state where the transition arc starts (the source), and
1. the label of the transition arc, and
1. the state the transition arc leads to (the target or goal).

We can expand our `fsa` class with a helper function `update` whose job it is to add such triples to the transitions dictionary.

In [ ]:
class fsa:
    def __init__(self, initial=0, final=set(), transitions={}):
        """Class for finite-state automata.
        
        The transitions must be
        1) a dictionary of the form
        {current_state: {arc_label: new_state}},
        or
        2) a list of tuples of the form
        (source, label, goal).
        """
        self.I = initial
        self.F = final
        # process transitions based on its type
        if isinstance(transitions, dict):
            # transitions is a dictionary, use as is
            self.T = transitions
        else:
            self. T = {}
            for t in transitions:
                self.update(t)
        

    def update(self, trans):
        """Add (source, label, goal) triples to transitions dictionary."""
        source, goal, label = trans
        self.T[source] = self.T.get(source, {})
        self.T[source][label] = goal
            

    def accepts(self, sentence):
        """Test if FSA accepts sentence."""
        # set current state to initial state
        cs = self.I
        # iterate over sentence and follow along in automaton
        for word in sentence:
            cs = self.T.get(cs, {}).get(word)
            if cs is None:
                return False
        # did we make it to a final state?
        return True if cs in self.F else False

This code expands `__init__` so that it first checks the type of `transitions`.
If it is a dictionary, it gets used as the value for `self.T` as is.
Otherwise, the helper method `update` is called on each member of `transitions`.
This makes it possible for the user to pass in transitions as a list of triples.

In [ ]:
from pprint import pprint

trans = [(0, "Adrian", 1),
         (1, "Rocky", 0)]
movie_quote = fsa(initial=0, final={0, 1}, transitions=trans)

pprint(movie_quote.T, width=1)

In fact, a set would have worked just as well.

In [ ]:
from pprint import pprint

trans = {(0, "Adrian", 1),
         (1, "Rocky", 0)}
movie_quote = fsa(initial=0, final={0, 1}, transitions=trans)

pprint(movie_quote.T, width=1)

Ironically, a dictionary with the transition triples as keys would also work, but does not do what we want.

In [ ]:
from pprint import pprint

trans = {(0, "Adrian", 1): "irrelevant",
         (1, "Rocky", 0): "irrelevant"}
movie_quote = fsa(initial=0, final={0, 1}, transitions=trans)

pprint(movie_quote.T, width=1)

That's because `isinstance(trans, dict)` holds in this case, so `trans` is passed through without being processed by the `update` method.
This shows that our sanity checks still let through quite a bit of crud.

## Tightening the checks

One radical solution to the problem with the previous example is to remove the passthrough clause for dictionaries.
Then the user can only use containers of tuples to specify the automaton's transitions.
The cell below shows the relevant fragment of the `fsa` class with these modifications.

In [ ]:
class fsa:
    def __init__(self, initial=0, final=set(), transitions={}):
        """Class for finite-state automata.
        
        The transitions must be a list of tuples of the form
        (source, label, goal).
        """
        self.I = initial
        self.F = final
        # process transitions into dictionary
        self. T = {}
        for t in transitions:
            self.update(t)
        

    def update(self, trans):
        """Add (source, label, goal) triples to transitions dictionary."""
        source, goal, label = trans
        self.T[source] = self.T.get(source, {})
        self.T[source][label] = goal

However, this actually widens how transitions can be specified.
Just consider our weird dictionary example from before.

In [ ]:
from pprint import pprint

trans = {(0, "Adrian", 1): "irrelevant",
         (1, "Rocky", 0): "irrelevant"}
movie_quote = fsa(initial=0, final={0, 1}, transitions=trans)

pprint(movie_quote.T, width=1)

Paradoxically, this now produces the correct kind of dictionary.
That's because we no longer check the type of `transitions`.
As long as it is iterable, the `for`-loop will pass each argment into the `.update` method.
In the case of a dictionary, `for` iterates over the keys by default.
So instead of limiting the input to sets or lists of tuples, we've allowed even more stuff.

That's not necessarily a bad thing.
As long as what gets fed into `update` is a triple of the right kind, it doesn't really matter what container those triples came in.
But without further checks this can still produce some unexpected problems, as the next code snippet illustrates.

In [ ]:
from pprint import pprint

movie_quote = fsa(initial=0, final={0, 1}, transitions=(0, "Adrian", 1))

pprint(movie_quote.T, width=1)

This input crashes the whole program!
That's because we passed in a single transition without a container.
But transitions, by virtue of being containers, are iterable, so the `for`-loop passes each component of the transition into `update`.
Instead of a triple, `update` now receives only the first component of the triple.
But the line `source, goal, label = trans` presupposes the presence of a triple, causing the code to crash.

Perhaps even more deviously, the lack of checks allows some inputs to pass that really shouldn't.
Whenever each component of a transition triple can be decomposed into three smaller units, `update` won't complain, but we'll get a completely nonsensical automaton.

In [ ]:
from pprint import pprint

movie_quote = fsa(initial=0, final={0, 1}, transitions=("say", "Bob", "end"))

pprint(movie_quote.T, width=1)

This shows just how devious the effects of faulty input can be.
So let's add some basic checks to make sure that such cases of insanity are avoided.
Since we only care about the shape of the transition triples, we'll put all these checks into the `update` method.

In [ ]:
class fsa:
    def __init__(self, initial=0, final=set(), transitions={}):
        """Class for finite-state automata.
        
        The transitions must be a list of tuples of the form
        (source, label, goal).
        """
        self.I = initial
        self.F = final
        # process transitions into dictionary
        self. T = {}
        for t in transitions:
            self.update(t)
        

    def update(self, trans):
        """Add (source, label, goal) triples to transitions dictionary."""
        if isinstance(trans, tuple) and len(trans) == 3:
            source, goal, label = trans
            self.T[source] = self.T.get(source, {})
            self.T[source][label] = goal
        else:
            print(f"Warning: {trans} is not a valid transition triple!")
            print("Skipping processing...")

This will still gobble up the odd dictionary specification, but will choke in the previous case where just a single  transition was passed in without a container.

In [ ]:
from pprint import pprint

trans = {(0, "Adrian", 1): "irrelevant",
         (1, "Rocky", 0): "irrelevant"}
movie_quote = fsa(initial=0, final={0, 1}, transitions=trans)

pprint(movie_quote.T, width=1)

In [ ]:
from pprint import pprint

movie_quote = fsa(initial=0, final={0, 1}, transitions=("say", "Bob", "end"))

pprint(movie_quote.T, width=1)

## Stopping execution with `assert`

The solution above works well enough as long as there actually is somebody sitting in front of the screen who reads our warning messages.
But we can't always rely on that.
Sometimes, it's better to say "Damn it, to much has already gone wrong here, abort! I repeat, abort!".
That's what Python's `assert` command is for.

This command is very easy to use:

```python
# abort if some_boolean is False
assert some_boolean
```

This will cause Python to stop the program is `some_boolean` is not `True`.
We may optionally add an error message.

```python
# abort if some_boolean is False
assert some_boolean, "some error message"
```

Here's a concrete example:

In [ ]:
assert 4 == 5, "4 does not equal 5"

By converting our `if` test in `update` to a sequence of `assert` statements, we enforce an explicit abortion point in the code: if `update` gets some input that can't possibly be a transition triple, stop processing.

In [ ]:
class fsa:
    def __init__(self, initial=0, final=set(), transitions={}):
        """Class for finite-state automata.
        
        The transitions must be a list of tuples of the form
        (source, label, goal).
        """
        self.I = initial
        self.F = final
        # process transitions into dictionary
        self. T = {}
        for t in transitions:
            self.update(t)
        

    def update(self, trans):
        """Add (source, label, goal) triples to transitions dictionary."""
        assert isinstance(trans, tuple), "Each transition must be a tuple"
        assert len(trans) == 3, "Each transition must have exactly 3 components"
        source, goal, label = trans
        self.T[source] = self.T.get(source, {})
        self.T[source][label] = goal

In [ ]:
from pprint import pprint

print("Code before the faulty pessage is run just fine.")

movie_quote = fsa(initial=0, final={0, 1}, transitions=("say", "Bob", "end"))

print("But this won't be executed anymore because Python stops halfway through the fsa-construction.")
pprint(movie_quote.T, width=1)

The `assert` command is a very powerful safeguard.
It makes everything grind to a screeching halt.
But this also means it shouldn't be used lightly.
Some errors can be fixed on the fly with smart coding, e.g. by providing default values.
Fixing errors comes with the risk of unwanted assumptions, though.
Suppose, for instance, that the user specifies only two transitions, `(0, "a", 1)` and `(2, "b", 3)`.
If our only initial state is `0`, then there is no way to reach `2`.
Is this a user error, or a deliberate decision?
Should we check for such errors, and if so, should we just print a warning or use `assert` to halt the whole program?

These are difficult questions, which is why sanity checking user input is very tricky.
There's several rules of thumbs, but they all contradict each other.

1. Don't go crazy with asserts.
   The more asserts you put in place, the less flexible your code becomes.
   For instance, `assert isinstance(transitions, set)` serves no purpose if a list works just as well.
   
1. Don't make hidden fixes.
   For instance, it might be tempting to just skip ill-formed transition triples while processing the rest.
   But then the automaton will behave very differently, and the user might not notice that anything has gone wrong.
   
1. Don't expect much of the user.
   Any mistake that one could make in specifying the input **will** be made.
   Try to anticipate them and put appropriate checks, fixes, and warnings in place.
   
Yeah, it's tricky.
For our `fsa`, one reasonable solution might look as follows:

1. Keep the `assert` lines inside `update` since there is no easy way to figure out what the intended input was.
1. Add an additional `sanity_check` method that checks for a few telltale signs of malformed user input.
   This includes:
   - There is no transition from the initial state(s) to any of the other states.
   - There is no transition to a final state.
   - Some states are unreachable.
1. Add an optional `verbose` parameter to `__init__` that, when set to `True`, displays the specified automaton as a graph.

As you an imagine, options 2 and 3 take quite a bit of work, so we won't try to tackle them here.
In the real world, though, we don't have the luxury of skipping sanity checks if they're too much work.
And make no mistake, error checking is a major component of programming that must not be taken lightly.
It's the difference between a hackjob and production-ready software.

## Bullet point summary

- Sanity checks are an important part of writing robust and reliable programs.
- For simple cases, you can use `assert` to stop the program when unrecoverable errors are encountered.

```python
# abort unless some_condition holds
assert some_condition, "error message that some condition was not satisfied"
```